<h1> Deep Q-learning for Flappy Bird </h1>
    
    
    Introduction/Background:

This project aims to study and select the best model of deep-q-learning in training agent to play the game "Flappy Bird", by first comparing performance done by different model, then choosing the best model, experience replay, and optimise it by experimenting the reward received each episode under different hyperparameters, ranging from initial epsilon, to learning rate, and memory size used to update the 3 layer convolution fully connected neural network. Agent first receive environment information through image per frame, epsilon greedy algorithm is then used to select agent's action (exploration or exploitation) in the next frame. The outcome, action taken, current frame information and next frame information after taking the action, is then stored into the memory batch which, is used to update the neural network, allowing the agent to learn how to "play the game".

    Data sources:

There is no pre-existing data source in our project, instead our agent learns while playing the game and gets data generated from the environment at each time step as an image, then the agent studies the bird position and pipe position. In other words, our agent is only given raw pixel information and will learn the environment thus predicitng the best action to take. After performing an action, a new image, reward and a boolean of game terminated or not, is being generated.

The game base we used include the game base *flappy_bird.py*, which is where the agent plays the game, and images in *assets\sprites* folder, which helps to display the game. These are obtained from this site: https://github.com/uvipen/Flappy-bird-deep-Q-learning-pytorch, written by uvipen

- The *next_frame()* funciton under *flappy_bird.py* takes in a number, 0 means no action, 1 means jump, and returns 3 variables: state as image, reward as float, terminal as boolean. 

- State is the image after action applied

- Reward is 0.1 if bird is alive, 1 if pass through pipe, -1 if dead.

- Terminal is TRUE if bird is dead, else FALSE


    Data Analysis:

The resulting image from flappy_bird.py is put to pre_process.py and used as our input data to network after pre processing. The values at the final output layer represent the Q values given the input state for each valid action.

A short summary of important classes and variables:
- Variables:
    - LEARNING_RATE     --- learning rate $\eta$, represents how important new data are in comparison to existing data when performing backward propagation
    - MAX_ITER          --- max number of iteration the program runs
    - MAX_EXPERIENCE    --- max number of experience stored
    - DISCOUNT_FACTOR   --- discount rate of future rewards
    - BATCH_SIZE        --- sample size when extracting batch from stored experiences
    - INITIAL_EPSILON   --- initial epsilon, probability of choosing a random action, used in the epsilon-greedy algorithm
    
    - state             --- stored as an image
    - next_state        --- stored as an image
    - action            --- stored as [1, 0] (no jump) or [0, 1] (jump)
    - reward            --- stored as float (0.1 or 1 or -1)
    - terminal          --- stored as boolean, bird dead = TRUE, else FALSE
- Classes:
    - *ThreeLayerConvModel* --- stores the 3 layer convolutional fully connected neural network
    
    - *Memory* --- stores the most recent gameplay state, reward, terminal into the class, if exceeds MAX_EXPERIENCE, drop the oldest memory
    - *FlappyBird* --- game base which takes in a value (action) and returns state, reward, terminal, written by uvipen (refer to link above)

In [ ]:
import os
import shutil
import torch.optim as optim
import torch
import torch.nn as nn
import numpy as np
from src.memory import *
from src.convModel import *
from src.utils import *
from src.flappy_bird import *
import random
from matplotlib import pyplot as plt
import time as t

    Methods: 
The *train_model()* sub is used to train the agent in playing the game

Steps:
1.  Generate initial state (get starting image by passing in *action = 0* into *game.next_frame()*)
2.  Perform random action or use prediction from the neural network, with the aids of epsilon greedy algorithm.
3.  Epsilon greedy algorithm has initial epsilon of 0.2, meaning initially 20% of the chance to select a random action (exploration). As the number of iteration increases, the chance the select a random action decreases linearly 
$$\text{new epsilon} = \text{initial epsilon} \times (1 - \frac{\text{number of iterations passed}}{\text{MAX ITERATION}})$$
4.  The action generated in step 3 is then inputted into *game.next_frame(action)*, which gives information about next frame: next_state, reward, terminal
5.  These information are then stored into class *Memory*, and the model is then updated (see below for detailed breakdown)
6.  A model is saved every 100000 iterations, and a graph is printed everytime the bird dies, showing how long the bird survived each gameplay

In [ ]:
# train_model will train model 2 million times and save model and graph every 100k times
"""
Method(s) or Model(s) (4 marks) :
Make use of model (s), models are appropriate and clearly applied.
"""
def train_model():
    
    # action to be taken by agent
    # [1, 0] means no action
    # [0, 1] means jump
    action = torch.zeros(2, dtype=torch.float32)
    
    # used to keep count number of iterations passed (number of frames)
    iter = 0

    # getting initial state
    state_image, reward, terminal = game.next_frame(action[1])
    
    # pre-processing on image, aiming to reduce image size and convert image to monochrome
    state = pre_processing(state_image)
    state = torch.cat((state, state, state, state)).unsqueeze(0)
    alive_stat = []
    alive_time = 0
    save_path = "trained_models"
    graph_path = "graph"
    
    # training the agent
    while iter < MAX_ITER:
        
        iter = iter + 1
        q_value = model(state)[0]
        
        action = torch.zeros(2, dtype=torch.float32)
        if torch.cuda.is_available():
            action = action.cuda()
        
        # Epsilon-Greedy implementation
        epsilon = INITIAL_EPSILON * (1 - iter / MAX_ITER)
        u = random.random()
        random_action = False if epsilon < u else True
        index = torch.argmax(q_value).item()
        
        if random_action:
            print("Performed random action!")
            index = randint(0, 1)     
        action[index] = 1  

        # Perform action and get next state information
        next_state_image, reward, terminal = game.next_frame(action[1])
        # Pre_process image 
        next_state_image = pre_processing(next_state_image)
        next_state = torch.cat((state.squeeze(0)[1:, :, :], next_state_image)).unsqueeze(0)
        
        reward = torch.from_numpy(np.array([reward], dtype=np.float32)).unsqueeze(0)
        
        action = action.unsqueeze(0)
        
        # save replay
        memory.push(state, action, next_state, reward, terminal)
        
        state = next_state
        
        # update neural network
        update_model()

        # print information of each frame
        print("Iteration: {}/{}, Action: {}, Reward: {}, Q-value: {}".format(
            iter + 1,
            MAX_ITER,
            action[0],
            reward, torch.max(q_value)))

        # saving model every 100000 iteration
        alive_time += 1
        if(iter+1) % 100000 == 0:
            torch.save(model, "{}/flappy_bird_{}".format(save_path, iter+1))
        
        # plot how long agent survived
        if terminal:
            alive_stat.append(alive_time)
            plot_duration(alive_stat)
            alive_time = 0
        
        if(iter+1) % 100000 == 0:
            plt.savefig('{}/train_{}.jpg'.format(graph_path, iter+1))
    torch.save(model,"{}/flappy_bird".format(save_path))

This part updates the model by selecting a batch_size from memory, update the model using the memory along with the loss function

Steps:
1.  Retrieve a list of memory from *Memory* class with size min(length of memory, BATCH_SIZE), where BATCH_SIZE is a specified constant
2.  Compute loss function:
$$[r_t + \gamma \max_b{Q_w(s_{t+1}, b)} - Q_w(s_t, a_t)]^2 ---- (1)$$
3.  The *y_batch* computes the $r_t + \gamma \max_b{Q_w(s_{t+1}, b)}$ part of the loss function, however since there is a possibility of bird dying and game terminates, there we consider if terminal = TRUE, we only use $r_t$
4.  The *q_value* computes the $Q_w(s_t, a_t)$ part
5.  The loss function is then calculated by *loss = criterion(y_batch, q_value)*, with criterion set as Mean-Squared Error, or mathematically:
$$ Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \eta \times (1)$$
$$ \text{where } \eta \text{ is learning rate} $$
6.  Model is then updated based on the loss function using backward propagation *loss.backward()*

In [ ]:
def update_model():
    
    batch = memory.sample(BATCH_SIZE)
    # unpack batch
    state_batch = torch.cat(tuple(d[0] for d in batch))
    action_batch = torch.cat(tuple(d[1] for d in batch))
    next_state_batch = torch.cat(tuple(d[2] for d in batch))
    reward_batch = torch.cat(tuple(d[3] for d in batch))
    terminal_batch =[d[4] for d in batch]
    
    if torch.cuda.is_available(): 
        state_batch = state_batch.cuda()
        action_batch = action_batch.cuda()
        reward_batch = reward_batch.cuda()
        next_state_batch = next_state_batch.cuda()
        terminal_batch = terminal_batch.cuda()

    next_action_batch = model(next_state_batch)
    
    # if dead, rj, otherwise r_j + gamma*max(Q_t+1)
    y_batch = torch.cat(tuple(reward_batch[i] if terminal_batch[i]
                                  else reward_batch[i] + DISCOUNT_FACTOR * torch.max(next_action_batch[i])
                                  for i in range(len(batch))))

    
    q_value = torch.sum(model(state_batch) * action_batch, dim=1)

    optimizer.zero_grad()

    # Detaching from the current graph so that it won't causes error in next iteration
    y_batch = y_batch.detach()

    # Calculate loss
    loss = criterion(q_value, y_batch)

    # Do backward pass
    loss.backward()
    optimizer.step()

This part plots the time agent survived every 100 episodes, used for reporting purpose (see diagrams used below)

In [ ]:
def plot_duration(duration):
    # Plot durations of episodes and average over last 100 episodes
    plt.figure(1)
    plt.clf()
    durations_t = torch.tensor(duration, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)

*test()* plays the game by using the trained model. A graph is being plotted and updated each time the bird dies, with y value as number of frames bird stays alive, and x value as number of games played. For every 100000 iteration, the graph is saved in jpg file format under folder *graph*

In [ ]:
# test will test 100000 iter and save the graph

def test():
    save_path = "trained_models"
    # Modify the model you want to test, keep "_" 
    model_number = "_100000"
    model = torch.load("{}/flappy_bird{}".format(save_path, model_number))
    model.eval()
    
    # Play the game using the trained model
    action = torch.zeros(2, dtype=torch.float32)
    state_image, reward, terminal = game.next_frame(action[1])
    state = pre_processing(state_image)
    state = torch.cat((state, state, state, state)).unsqueeze(0)
    alive_time = 0
    iter = 0
    graph_path = "graph"
    alive_stat = []
    while iter < MAX_ITER:
        q_value = model(state)[0]
        action = torch.argmax(q_value, 0)
        next_state_image, reward, terminal = game.next_frame(action)
        next_state_image = pre_processing(next_state_image)
        next_state = torch.cat((state.squeeze(0)[1:, :, :], next_state_image)).unsqueeze(0)
        state = next_state
        
        alive_time += 1
        
        # Add in data (how many frame bird survived) to the plot
        if terminal:
            alive_stat.append(alive_time)
            plot_duration(alive_stat)
            alive_time = 0
        
        # Graph saved
        if(iter+1) % 100000 == 0:
            plt.savefig('{}/test{}.jpg'.format(graph_path, model_number))
        iter += 1

    Result and Discussion:
Three models had been trained at the beginning to decide which one is the better approach
1.  Q-learning with 3-layer Convolutional Fully Connected Network (Q-learning is trained by setting MAX_EXPERIENCE and BATCH_SIZE as 1)
2.  Deep Q-Leraning Experience Replay with 3-layer Convolutional Fully Connected Network
3.  Double Q-Learning with 3-layer Convolutional Fully Connected Network

Their performance is then compared based on rewards they get and time used to train.

Q-Learning: 3 Layer Q-Learning

<img src="graph/train_100000.jpg" width="300"> 

Deep Q-Learning (Experience Replay): 3 Layer Convolutional Fully Connected Network 

<img src="train_norm.png" width="300"> 

Double Q-Learning: 3 Layer Convolutional Fully Connected Network

<img src="DDQ.png" width="300">

By analyzing the graph, we can see that experience replay is trained successfully as the reward it gains are increasing.

Double Q-Learning does get a high reward and achieved this faster than our first method(reward graph converges earlier), but the rewards drops significantly and remains low after around 1600 episodes. From research and analysis, this might be caused by overfitting the Q function. In this secenario, we have two sets of weights w and w1, generated from two netwroks network1 and targetnetwork. One of them is used for action selection and the other for evaluation. To solve this problem(for further experiments), we could test out different hyper-parameters and implement a learning rate decay methods that suits.

After the experiements, comparisons, and research, we've decided to use a 3-layer DQN experience replay model, which is more stable for taining purpose, faster, and have a generally good performance. However, we also noticed the importance of hyper-parameters to our model. So, in order to find the best performing model,we decided to alter the variables (Initial epsilon, learning rate, max experience, batch size) and compare the differences between each output, thus decides which is the best experience replay model:

Since the performance of experience replay is affected by some predetermined parameters, we decided to test out how each parameter affects the model performanc:

Training:

<img src="train.PNG" width="1500"> 

Testing:

<img src="test.PNG" width="1500">

*(For ease of analysis, we refer left most scenario as Scenario 1, and right most scenario as Scenario 4)*

*For analysis below, we will use Scenario 1 as the base scenario, and compare it against Scenario 2 - 4*

For the training diagram, we can see under Scenario 1 to Scenario 3, agents tend to train successfully, whereas for Scenario 4, agent fails to train. We believe Scenario 4 agent fails to train because the initial epsilon is too high, there is a 60% chance for agent to choose a random action!

Under Scenario 3, agent trains successfully however, the time taken to pass through 3000 episdoes (games) is almost doubled comparing to Scenario 1 and 2. This is because the model is updated every iteration and, each iteration we have to re-extract a much larger sample size from the stored memories thus, it requires more time per iteration.

Under Scenario 2, the agent seems to be training slightly better than the base secenario towards the end (roughly 2000+ episodes), however simply from the training diagram there are no strong indicators suggesting either model outperforms one another.



However, the testing diagram suggests a different result:

From the testing diagram, Scenario 4 is blanked out because the program crashed halfway as such, it is safe to conclude that Scenario 4 is not a good approach.

Scenario 3 does have a higher maximum reward obtained comparing to base scenario however, if we refer to the average line (orange line), both models have very similar average reward. The reason being is likely that there is already sufficient memory size and sample size taken under the base scenario, increasing both size will not affect the average reward significantly however, increases the time taken drastically

Scenario 2 on the other hand, shows that the agent is able to learn the environment very well, with the average reward roughly being 4 times higher than the base scenario! This is likely because the learning rate in the base scenario is too high that it is overfitting the model, as such it is safe to conclude that scenario 2 is the best model to use for deep q learning in this environment

To conclude, Experience replay generally is a good approach in reinforced learning however, the main weakness is that correct hyperparameters need to be chosen, and the correct hyperparameters can only be chosen after multiple trial and errors.

This is the code to train the agent

In [ ]:
LEARNING_RATE = 5e-5
MAX_ITER = 500000
MAX_EXPERIENCE = 50
DISCOUNT_FACTOR = 0.99
BATCH_SIZE = 30
INITIAL_EPSILON = 0.2

start_time = t.time()
iter = 0

# Constructing memory class
memory = Memory(MAX_EXPERIENCE)

criterion = nn.MSELoss()
game = FlappyBird()
model = ThreeLayerConvModel()
optimizer = optim.Adam(model.parameters(), LEARNING_RATE)
graph_path = "graph"
plt.ion()
train_model()
print(t.time() - start_time)
plt.ioff()
plt.show()

This is the code to test the trained model

In [ ]:
MAX_ITER = 100000
iter = 0
game = FlappyBird()

plt.ion()

test()
plt.ioff()
plt.show()